# Install and Import Dependencies

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

2023-11-28 23:57:38.387581: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# Calculate Angles

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# Footstep Counter

In [4]:
# initialize constants
STAGE_DOWN = 'DOWN'
STAGE_UP = 'UP'

In [5]:
cap = cv2.VideoCapture(0)

# intialize counter variables
counter = 5
stage = None
r_stage = STAGE_DOWN
l_stage = STAGE_DOWN

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            rhip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            rknee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            rankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            lhip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            lknee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            lankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            r_angle = calculate_angle(rhip, rknee, rankle)
            l_angle = calculate_angle(lhip, lknee, lankle)
            
            # Visualize angle
            cv2.putText(image, str(r_angle), 
                           tuple(np.multiply(rknee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(l_angle), 
                           tuple(np.multiply(lknee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )           
            
            # counter logic
            if stage == None:
                if r_angle < 160 and r_stage == STAGE_DOWN:
                    r_stage = STAGE_UP
                    stage = STAGE_UP
                if l_angle < 160 and l_stage == STAGE_DOWN:
                    l_stage = STAGE_UP
                    stage = STAGE_UP
            elif stage == STAGE_UP:
                if r_angle >= 170 and r_stage == STAGE_UP:
                    r_stage = STAGE_DOWN
                    stage = None
                    counter +=1
                if l_angle >= 170 and l_stage == STAGE_UP:
                    l_stage = STAGE_DOWN
                    stage = None
                    counter +=1
                                       
        except:
            pass
        
        # Render footstep counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)       
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        # exit the program by pressing "q" on the keyboard
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        # reset counter and stages by pressing "r" on the keyboard
        if cv2.waitKey(10) & 0xFF == ord('r'):
            counter = 0
            l_stage = STAGE_DOWN
            r_stage = STAGE_DOWN
            stage = None

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
